# Web Scraping

In [2]:
import requests

In [3]:
from bs4 import BeautifulSoup

In [4]:
import pandas as pd

In [5]:
movies=[]
base_url="https://www.imdb.com/list/ls050782187/"

response=requests.get(base_url)

if response.status_code == 200:
    soup=BeautifulSoup(response.content,'html.parser')
    
    movie_data=soup.find_all('div',class_="lister-item-content")
    
    for value in movie_data:
        title= value.h3.a.text
        year= value.h3.find('span',class_="lister-item-year text-muted unbold").text.strip('()')
        genre= value.find('span', class_='genre').text.strip()
        runtime = value.find('span', class_='runtime').text.strip()
        rating = value.find('span', class_='ipl-rating-star__rating').text.strip()
        description = value.find('p', class_='').text.strip()
        director = value.find_all('p')[2].find('a').text.strip()
        gross_tag = value.find('span', attrs={'name': 'nv'})
        gross = gross_tag['data-value'] if gross_tag else None

        movies.append([title, year, genre, runtime, rating, description, director, gross])

In [7]:
while len(movies)<500:
    next_page_link= soup.find('a',class_="flat-button lister-page-next next-page")
    
    next_page_url = 'https://www.imdb.com/list/ls050782187/?sort=list_order,asc&st_dt=&mode=detail&page=2' + next_page_link.get('href')
    
    response = requests.get(next_page_url)
    
    if response.status_code ==200:
        soup= BeautifulSoup(response.content,'html.parser')
        
        movie_data = soup.find_all('div',class_='lister-item-content')
        
        for value in movie_data:
            title = value.h3.a.text
            year = value.h3.find('span', class_='lister-item-year').text.strip('()')
            genre = value.find('span', class_='genre').text.strip()
            runtime = value.find('span', class_='runtime').text.strip()
            rating = value.find('span', class_='ipl-rating-star__rating').text.strip()
            description = value.find('p', class_='').text.strip()
            director = value.find_all('p')[2].find('a').text.strip()
            gross_tag = value.find('span', attrs={'name': 'nv'})
            gross = gross_tag['data-value'] if gross_tag else None

            movies.append([title, year, genre, runtime, rating, description, director, gross])
    else:
        print(f"Failed to retrieve data from next page: {next_page_url}")
        break
            

In [8]:
df = pd.DataFrame(movies, columns=['Title', 'Year', 'Genre', 'Runtime', 'Rating', 'Description', 'Director', 'Gross'])

pd.set_option('display.max_colwidth', None)  # Display full content of columns
pd.set_option('display.expand_frame_repr', False)  # Don't wrap lines in the display
pd.set_option('display.width', None)
    # Print DataFrame
print(df)

                                              Title      Year                       Genre  Runtime Rating                                                                                                                                                                                              Description              Director    Gross
0                                     The Godfather      1972                Crime, Drama  175 min    9.2                                                                                      The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.  Francis Ford Coppola  1999865
1                          The Silence of the Lambs      1991      Crime, Drama, Thriller  118 min    8.6                                      A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.        Jonathan D